In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from tqdm import tqdm

### 데이터 압축해제

In [ ]:
# 라벨링데이터 압축 풀기
%cd /content/drive/MyDrive
!unzip -qq "/content/drive/MyDrive/determining_code_similarity_AI_competition-20220612T174726Z-001.zip"

In [ ]:
# # 라벨링데이터 압축 풀기
# %cd /content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label
# !unzip -qq "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label_data.zip"

In [ ]:
# # 원천데이터 압축 풀기
# %cd /content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image
# !unzip -qq "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image_data.zip"

In [ ]:
# 데이터 개수 확인
import os

def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)
	
dir_path_label = "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label"
dir_path_image = "/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image"
cnt_label = 0
cnt_image = 0
fcnt_label = {}
fcnt_image = {}

for (root, directories, files) in tqdm(os.walk(dir_path_label)):
  for d in directories:
    f_l = get_files_count(os.path.join(root, d))
    fcnt_label[d] = f_l
    cnt_label += f_l

for (root, directories, files) in tqdm(os.walk(dir_path_image)):
  for d in directories:
    f_i = get_files_count(os.path.join(root, d))
    fcnt_image[d] = f_i
    cnt_image += f_i

print(fcnt_label)
print(fcnt_image)

for key in fcnt_label.keys():
  if fcnt_label[key] != fcnt_image[key]:
    print(fcnt_label[key],",",fcnt_image[key])
print("전체 레이블 수: ",cnt_label,"\t전체 이미지 수: ",cnt_image)

In [ ]:
# # 오류났을 경우 폴더 삭제
# import os
# import shutil

# dir_path = "/content/drive/MyDrive/deep-learning-from-scratch-master"

# if os.path.exists(dir_path):
#     shutil.rmtree(dir_path)
# os.mkdir("/content/drive/MyDrive/deep-learning-from-scratch-master")

### 데이터 전처리

In [ ]:
import os
import numpy as np
import pandas as pd
import json #json을 파싱하기 위해
import re
from sklearn.model_selection import train_test_split

In [ ]:
# DataFrame 출력 사이즈 조정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', 50)

In [ ]:
def file_to_json(f):
    data = json.load(f)
    yield data["이미지 정보"]
    yield data["데이터셋 정보"]

In [ ]:
dir_path = '/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label'
images = []
datasets = []

for (root, directories, files) in os.walk(dir_path):
    for file in tqdm(files):
        file_path = os.path.join(root, file)
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            g = file_to_json(f)
            image_data = next(g)
            images.append(image_data)
            dataset_data = next(g)
            datasets.append(dataset_data)

In [ ]:
# 이미지 정보 DataFrame
image_df = pd.DataFrame(images)
image_df.drop(["이미지 파일명"],axis=1, inplace=True)
image_df.rename(columns={"이미지 식별자":"파일 번호"}, inplace=True)

In [ ]:
image_df.tail()

In [ ]:
image_df.to_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image_df.csv', sep=',', na_rep='NaN')

In [ ]:
# 데이터셋 정보 DataFrame
pre_dataset_df = pd.DataFrame(datasets)
dataset_json = pre_dataset_df.to_json(orient="table")
dataset_json = json.loads(dataset_json)
dataset_df = pd.json_normalize(data=dataset_json['data'])
dataset_df.drop(["index","파일 생성일자","파일 이름",
                 "데이터셋 상세설명.렉트좌표.아우터","데이터셋 상세설명.렉트좌표.하의","데이터셋 상세설명.렉트좌표.원피스","데이터셋 상세설명.렉트좌표.상의",
                "데이터셋 상세설명.폴리곤좌표.아우터","데이터셋 상세설명.폴리곤좌표.하의","데이터셋 상세설명.폴리곤좌표.원피스","데이터셋 상세설명.폴리곤좌표.상의"],
                axis=1, inplace=True)
dataset_df.rename(columns={"데이터셋 상세설명.라벨링.스타일":"스타일","데이터셋 상세설명.라벨링.아우터":"라벨링_아우터","데이터셋 상세설명.라벨링.하의":"라벨링_하의","데이터셋 상세설명.라벨링.원피스":"라벨링_원피스","데이터셋 상세설명.라벨링.상의":"라벨링_상의"}, inplace=True)

In [ ]:
dataset_df.head()

In [ ]:
dataset_df.to_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/dataset_df.csv', sep=',', na_rep='NaN')

In [ ]:
# # '아우터' 프로파일링 파일 만들어보기
# import pandas as pd
# import pandas_profiling

# outer_df = dataset_df[['파일 번호','라벨링_아우터']]
# outer_csv = outer_df.to_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/outer.csv', sep=',', na_rep='NaN')
# outer = pd.read_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/outer.csv',encoding='utf-8-sig')
# pr = outer.profile_report() # 프로파일링 결과 리포트를 pr에 저장
# pr

In [ ]:
dataset_df = pd.read_csv('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/dataset_df.csv',encoding='utf-8-sig')
dataset_df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
dataset_df.head()

In [ ]:
# '스타일'열 정리
select = ['페미닌', '스포티', '레트로', '로맨틱', '섹시', '밀리터리', '스트리트', '모던', '프레피']
for i in tqdm(range(len(dataset_df))):
  styles = list(eval(dataset_df.loc[i]['스타일'][1:-1]).values())
  for style in styles:
    if style not in select:
      styles.remove(style)
  dataset_df.at[i,'스타일'] = styles

In [ ]:
# image 파일들을 한 dir로 합치기
import shutil
import os

dir_path = '/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image'
for (root, directories, files) in os.walk(dir_path):
  for file in tqdm(files):
    file_path = os.path.join(root, file)
    new_path = shutil.move(file_path, dir_path)

In [ ]:
# label 파일들을 한 dir로 합치기
import shutil
import os

dir_path = '/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/label'
for (root, directories, files) in os.walk(dir_path):
  for file in tqdm(files):
    file_path = os.path.join(root, file)
    new_path = shutil.move(file_path, dir_path)

In [ ]:
for i in tqdm(range(len(dataset_df))):
  if dataset_df.loc[i]['스타일']==[]:
    num = dataset_df.loc[i]['파일 번호']
    os.remove('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/image/'+num+'.jpg')
    dataset_df.drop(index=i, inplace=True)
    image_df.drop(index=i, inplace=True)

In [ ]:
len(dataset_df)

In [ ]:
dataset_df

train_labels = []

In [ ]:
# 아우터
outer_df = dataset_df[['라벨링_아우터']]
outer_df

In [ ]:
outer_df = dataset_df[['파일 번호','라벨링_아우터']]
cond = outer_df.라벨링_아우터=='[{}]'
full = outer_df.loc[cond]
full

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/dataset_df.csv", encoding='utf-8-sig')
## 스타일:기타만 추출하기
# etc = df
# for r in etc.index:
#   if re.search('[가-힣]+', str(etc.loc[r,'스타일'])) is not None:
#     etc.drop(r, inplace=True)
# etc.head()

In [ ]:
dataset_df.drop(['Unnamed: 0','스타일'], axis=1, inplace=True)

In [ ]:
#######이거 그대로 주피터 노트북에서 실행하기

X = []
Y = []

for r in df.index:
  filename = df['파일 이름']
  img_path = f'이미지 경로/{filename}.jpg'

  # image data
  img = image.load_img(img_path, target_size=(800, 800), interpolation='nearest')
  img_tensor = image.img_to_array(img)
  img_tensor = preprocess_input(img_tensor)
  X.append(img_tensor)

  # label data
  categories = [0,0,0,0]
  for c,_ in enumerate(df.columns):
    if re.search('[가-힣]+', str(df.iloc[r,c])):
      categories[c] = 1
  Y.append(categories)
  
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,
                                                    random_state=100,
                                                    test_size=0.1,
                                                    stratify=Y,)  
data = (X_train, X_test, Y_train, Y_test)
# 파일로 저장
X_train.to_csv('/X_train.csv',index=False, encoding="utf-8")
X_test.to_csv('/X_test.csv',index=False, encoding="utf-8")
Y_train.to_csv('/Y_train.csv',index=False, encoding="utf-8")
Y_test.to_csv('/Y_test.csv',index=False, encoding="utf-8")

###모델 쌓기

In [ ]:
X_train = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/X_train.npy')
X_test = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/X_test.npy')
Y_train = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/Y_train.npy')
Y_test = np.load('/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/Y_test.npy')

In [ ]:
import tensorflow as tf
import keras
from keras import models, layers, Input, optimizers, initializers, regularizers, metrics, losses
from keras.models import Model, Sequential
from keras.applications.resnet_v2 import ResNet50V2, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = ResNet50V2(weights=None, input_tensor=Input(shape=(100,100,3)), classes=4)

In [ ]:
input_shape = (100, 100, 3)
drop_rate = 0.5

In [ ]:
model.compile(optimizer='adam',
              loss=losses.categorical_crossentropy,
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=128,
                    epochs=50,
                    validation_data=(X_test, Y_test))
model.save("/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/model2")

In [ ]:
model = keras.models.load_model("/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/model2")

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history["val_accuracy"]

plt.plot(range(1, len(acc)+1), acc, label="train")
plt.plot(range(1, len(acc)+1), val_acc, label="train")
plt.legend()

In [ ]:
img_path = '/content/drive/MyDrive/k-fashion_tagging_AI/k-fashion_dataset/KakaoTalk_20220623_023658545.jpg'
img = image.load_img(img_path, target_size=(100, 100), interpolation='nearest')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# print('Predicted:', decode_predictions(preds, top=3)[0])
print(np.argmax(preds))

In [ ]:
model.summary()

#시도

In [ ]:
import tensorflow as tf
from keras import models, layers, Input, optimizers, initializers, regularizers, metrics
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Dropout, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
import matplotlib.pyplot as plt

In [ ]:
model = Sequential()
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(Conv2D(32, (input_h, input_w), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dense(1, input_dim=3, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(1, input_dim=3, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Activation("softmax"))

In [ ]:
import tensorflow as tf
from keras import models, layers, Input, optimizers, initializers, regularizers, metrics
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
import matplotlib.pyplot as plt

In [ ]:
class ResidualBlock(Model):
  def __init__(self, channel_in=64, channel_out=256):
    super().__init__()
    channel = channel_out // 4

    self.conv1 = Conv2D(channel, kernel_size=(1,1), padding="same")
    self.bn1 = BatchNormalization()
    self.av1 = Activation(tf.nn.relu)
    self.conv2 = Conv2D(channel, kernel_size=(3,3), padding="same")
    self.bn2 = BatchNormalization()
    self.av2 = Activation(tf.nn.relu)
    self.conv3 = Conv2D(channel_out, kernel_size=(1,1), padding="same")
    self.bn3 = BatchNormalization()
    self.shortcut = self._shortcut(channel_in, channel_out)
    self.ass = Add()
    self.av3 = Activation(tf.nn.relu)

  def call(self, x):
    h = self.conv1(x)
    h = self.bn1(h)
    h = self.av1(h)
    h = self.conv2(x)
    h = self.bn2(h)
    h = self.av2(h)
    h = self.conv3(x)
    h = self.bn3(h)
    shortcut = self.shortcut(x)
    h = self.add([h, shortcut])
    y = self.av3(h)

    return y

  def _shortcut(self, channel_in, channel_out):
    if channel_in == channel_out:
      return lambda x : x
    else:
      return self._projection(channel_out)
    
  def _projection(self, channel_out):
    return Conv2D(channel_out, kernel_size=(1,1), padding="same")

In [ ]:
class ResModel(Model):
  def __init__(self, input_shape, output_dim):
    super().__init__()

    self._layers = [
        # Conv1
        Conv2D(64, input_shape=input_shape, kernel_size=(7,7), strides=(2,2), padding="same"),
        BatchNormalization(),
        Activation(tf.nn.relu),
        MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="same"),
        # Conv2
        ResidualBlock(64, 256),
        [
            ResidualBlock(256, 256) for _ in range(2)
        ],
        # Conv3
        Conv2D(512, kernel_size=(1,1), strides=(2,2)),
        [
            ResidualBlock(512, 512) for _ in range(4)
        ],
        # Conv4
        Conv2D(1024, kernel_size=(1,1), strides=(2,2)),
        [
            ResidualBlock(1024, 1024) for _ in range(6)
        ],
        # Conv4
        Conv2D(1024, kernel_size=(1,1), strides=(2,2)),
        [
            ResidualBlock(2048, 2048) for _ in range(3)
        ],
        GlobalAveragePooling2D(),
        Dense(1000, activation=tf.nn.relu),
        Dense(output_dim, activation=tf.nn.softmax)
    ]

  def call(self, x):
    for layer in self._layers:
      if isinstance(layer, list):
        for l in layer:
          x = l(x)
      else:
        x = layer(x)
    return x

In [ ]:
# 입력층 정의
input = Input(shape=(,))

# 첫번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
conv = Conv2D(16, activation="relu")(inputA)
conv = Dense(8, activation="relu")(x)
conv = Model(inputs=inputA, outputs=x)

# 두번째 입력층으로부터 분기되어 진행되는 인공 신경망을 정의
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(8, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

# 두개의 인공 신경망의 출력을 연결(concatenate)
result = concatenate([x.output, y.output])

z = Dense(2, activation="relu")(result)
z = Dense(1, activation="linear")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape and normalize input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3


# use functional API to build cnn layers
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)

# classifier loss, Adam optimizer, classifier accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x_train,
          y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          batch_size=batch_size)

# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

In [ ]:
from transformers import AutoFeatureExtractor, ResNetForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])